# How to create a Bitcoin address

- Uses ECDSA keypair 
- Uses SHA-256 and RIPEMD-160 hashing algorithms
- Uses Base58Check encoding on the 25-byte binary bitcoin address


References: 
- https://en.bitcoin.it/wiki/Technical_background_of_version_1_Bitcoin_addresses
- https://stackoverflow.com/questions/443967/how-to-create-python-bytes-object-from-long-hex-string

### 1. Create a encryption key pair

In [1]:
from ecdsa import SigningKey, SECP256k1

sk = SigningKey.generate(curve=SECP256k1)
vk = sk.get_verifying_key()

sk_str = sk.to_string().hex()

In [2]:
sk_str #Private

'3052733a3d8c2ae35221ca40837f9b37b4554d70777f7adfb31600d79d12ab23'

In [3]:
vk.to_string().hex()

'75b4f2d6a453667f13e8bdef7c250c7fd908cc13ecba988eedf638168869b3938aa95030bf95bdd6e554faad51c314e2a9d5703d5b9b7531c961c2d326e9b68c'

### 2. Performing Hashing algorithms

In [4]:
# Stage 1: 

private_key = 'fc40f69bd646005c31b04d6f098a77b0f46371ac2b07ba0e6064f1f94a587b43'
public_key = 'c90c481484a15f3d7feff6dcf5f2e73a91790e73ea519ba03f1d2054fb899d73e14368230222c74caf624b181e319ec1103b2360aa0ec53e66b6a092f9d435ff'

In [5]:
# Stage 2: Perform first hash using SHA256
from Crypto.Hash import SHA256

hash_obj = SHA256.new(str.encode(public_key))
hash_1 = hash_obj.digest()

hash_1.hex() # len of 64

'9bfe85ff58fad79bba0cdee3737abfba8d3e29020a48eb0d06c6018ced6e18ae'

In [6]:
# Stage 3: Perform 2nd hash using SHA RIPEMD-160 

from Crypto.Hash.RIPEMD import RIPEMD160Hash

RIPEMD160_hash = RIPEMD160Hash.new(hash_1)
hash_2 = RIPEMD160_hash.digest()

hash_2.hex()

'9c1185a5c5e9fc54612808977ee8f548b2258d31'

In [7]:
# Stage 4: Add version byte in front of RIPEMD-160 hash (0x00 for Main Network)

hash_3 = "00" + hash_2.hex()

hash_3

'009c1185a5c5e9fc54612808977ee8f548b2258d31'

In [8]:
# Stage 5: Perform SHA-256 Hash on extended RIPEMD-160 result

hash_4 = SHA256.new(str.encode(hash_3)).digest()

hash_4.hex()

'0dda4ac0c0cc11b8a40f4c46c82e337cf41f3450fd29301de92a76157e611996'

In [9]:
# Stage 6: Perform SHA-256 Hash on the previous result 

hash_5 = SHA256.new(hash_4).digest()

hash_5.hex()

'7ee6db297c1e7330e01810f1c04eef918404910750694f4f5cb665ca1e84a0cd'

In [10]:
# Stage 7 : Take first 4 bytes to create the address checksum

checksum = hash_5[:4].hex()

checksum

'7ee6db29'

In [11]:
# Stage 8: Add the checksum to the extended RIPEMD-160 at the end of stage 4

bit_address = hash_3 + checksum 

print("25-byte binary bitcoin address:\n'{}'".format(bit_address))


25-byte binary bitcoin address:
'009c1185a5c5e9fc54612808977ee8f548b2258d317ee6db29'


In [12]:
bytes.fromhex(bit_address)

b'\x00\x9c\x11\x85\xa5\xc5\xe9\xfcTa(\x08\x97~\xe8\xf5H\xb2%\x8d1~\xe6\xdb)'

In [13]:
# Stage 9: Convert the result from a byte string into a base58 string
# using Base58Check encoding. This is the most commonly used bitcoin address format
import base58

address = base58.b58encode(bytes.fromhex(bit_address))

address

'1FEDQP2D5zmpRXRG5zQn7vYVs4JVHd59fE'

### 3. Create functions to generate the a bitcoin address

In [66]:
# TODO 